<a href="https://colab.research.google.com/github/sgr1118/GD-NLP-Theory/blob/main/GD1_text_data_process(220921).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1-1. 들어가며

[자연 언어 와 프로그래밍 언어, AI Study](https://dukeyang.tistory.com/2)

Q1. 자연어와 프로그래밍 언어는 형식언어 이론상 각각 어떤 문법을 따르고 있나요?
- 프로그래밍 : 처리가 쉬운 것이 중요하기 때문에, 문맥 자유 문법으로 설계, 문법의, 모호함이 없도록 개발
- 자연언어 : 세심한 설계 없이 자연 발생, 처리가 어려우며 문맥 의존 문법에 해당, 처리가 어려운 이유로, 일단 문맥 자유 문법을 기본으로 하고, 문맥 의존 요소를 처리하는 방법으로 처리 방향이 가고 있음.

Q2. 기계적으로 언어를 해석하는 파서를 만들었을 때 해석상 의미의 모호성이 생기지 않으려면 그 언어가 어떤 문법 규칙을 준수해야 하나요?
- 문맥자유 문법(Context-free Grammar)을 따르는 언어여야 모호성이 없이 파싱트리를 만들 수 있습니다.

[Announcing SyntaxNet, Google AI Blog](https://ai.googleblog.com/2016/05/announcing-syntaxnet-worlds-most.html)

Q3. 윗글에서 소개된 "Alice drove down the street in her car." 라는 문장은 두가지로 해석될 수 있는 모호성을 가지고 있습니다. 어떻게 해석이 가능한가요?
- 앨리스는 자기 차를 운전하여 거리를 달려갔다., 앨리스는 자기 차 안에 있는 거리를 운전해 달려갔다.

Q4. 문법적으로 가능한 위 두 가지 해석 중 어느 것이 맞는지를 파악하려면 어떤 것이 필요하다고 하나요?
-  require real world knowledge!, 현실 세계에 대한 지식이 필요하다.

자연어는 문맥에 따라 얼마든지 해석이 달라질 수 있다.

오늘 다루게 될 내용은 자연어를 그렇게 단어의 의미 단위로 쪼개 내는 작업 자체도 만만치 않다는 것을 보여줄 것입니다. 자연어를 의미 단위로 쪼개는 토큰화(Tokenization) 기법은 자연어처리 모델의 성능에 결정적 영향을 미칩니다. 토크나이저가 문장을 단어로 쪼개는 방식에 따라 같은 문장도 완전히 다른 워드 벡터 리스트가 되기 때문입니다.

1. 학습 목표

(1) 분산 표현에 대한 직관적 이해를 얻는다.

(2) 문장 데이터를 정제하는 방법을 배운다.

(3) 토큰화의 여러 가지 기법들을 배운다.

(4) 단어 Embedding을 구축하는 방법에 대해 가볍게 맛본다.

1-2. 전처리 : 자연어의 노이즈 제거

자연어의 정의는 서론에서 배웠습니다. 그렇다면 자연어에는 노이즈가 많다는 말이 이해가 가실까요? 예시를 하나 보겠습니다

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-1-L-2.max-800x600_Hcen3qw.png)
<center>자연어 노이즈 예시</center>

위 예시와 같은 채팅 데이터는 띄어쓰기와 맞춤법, 약어 사용 등에서 문제가 많다. 이런 것들이 전부 자연어에서 노이즈가 된다.  따라서 아직은 소설책이나 신문 기사같이 맞춤법이 비교적 정확하고 노이즈가 적은 데이터를 사용하는 것이 일반적입니다. 혹은 여러 가지 테크닉으로 노이즈를 제거한 후 사용하는 방법이 있죠!

그럼 채팅 데이터와 같이 이상적이지 않은 데이터에서 발견할 수 있는 노이즈에는 어떤 유형들이 있을까요?

불완전한 문장으로 구성된 대화의 경우
한 문장씩 주고 받는 대화와 달리 메신저는 한 문장을 여러 번에 나눠 전송하거나 여러 문장을 한 번에 전송하는 경우가 있습니다.

예1) A: "아니아니" "오늘이 아니고" "내일이지" / B: "그럼 오늘 사야겠네. 내일 필요하니까?"

문장의 길이가 너무 길거나 짧은 경우
아주 짧은 문장은 의미가 없을 수 있고, 대체로 사용빈도가 높은 리액션에 해당하는 경우가 많아서 언어 모델을 왜곡시킬 우려가 있기 때문에 제외해 주는 게 좋습니다.

예1) A: "ㅋㅋㅋ", "ㅠㅠㅠ"

아주 긴 문장은 대화와는 관계가 없는 문장일 수 있습니다.

예2) A: "이 편지는 영국에서부터 시작되어…"

채팅 데이터에서 문장 시간 간격이 너무 긴 경우
메신저는 누가 자판을 치는지 모르기 때문에 서로의 말이 얽히게 됩니다. 따라서 서로의 말의 텀이 짧으면 그것은 대화가 아니라 서로 할말만 하는 상태일 수 있습니다.

예1) A: "겨울왕국2" / B: "보러가자" / A: "엊그제 봤는데 잼씀" / B: "오늘 저ㄴ 아니 ㅡㅡ"

혹은 말의 텀이 너무 길다면 그것은 연속된 대화로 보기 어렵습니다.

예2) A: "나 10만원만 빌려줄 수 있어?" / …… / B: "아 미안 지금 봤다 아직 필요해?"

바람직하지 않은 문장의 사용
욕설의 비율이나, 오타의 비율이 높은 문장은 자연어 모델 학습에 사용하지 않는 것이 좋습니다.





노이즈 유형 (1) 문장부호 : Hi, my name is john.

우리는 문장부호를 배웠으니 Hi, 가 Hi 와 , 의 결합인 것을 알지만 컴퓨터는 명시해 주지 않는다면 알파벳에 , 가 포함되어 있다고 생각할 수도 있겠죠? 문장부호를 단어와 분리하면 해결이 되는 상황이기 때문에 문장부호 양쪽에 공백을 추가하는 방법을 취합시다!

In [ ]:
def pad_punctuation(sentence, punc):
  for p in punc:
    sentence = sentence.replace(p, ' ' + p + ' ')
  return sentence

sentence = "Hi, my name is john."

print(pad_punctuation(sentence, [".", "?", "!", ","]))

# 단순히 문장부호 양옆에 공백을 추가해 주는 방식은 문제를 야기할 수도 있다.
# www.naver.com > www. naver. com 같이 구분되는 경우도 있음

Hi ,  my name is john . 


노이즈 유형 (2) 대소문자 : First, open the first chapter.




In [ ]:
sentence = "First, open the first chapter."

print(sentence.lower())
print(sentence.upper())

first, open the first chapter.
FIRST, OPEN THE FIRST CHAPTER.


노이즈 유형 (3) 특수문자 : He is a ten-year-old boy.

마지막인 노이즈 유형 (3) 특수문자입니다. ten-year-old와 seven-year-old, 그 외의 수많은 나이 표현들을 각각의 단어 취급을 해버리는 일이 벌어지겠죠. 그런 불상사를 방지하기 위해 특수문자를 제거하려 합니다. 하지만 무한한 특수문자를 모두 정의하여 제거할 수는 없겠죠? 따라서 우리는 사용할 알파벳과 기호들을 정의해 이를 제외하곤 모두 제거하도록 하겠습니다.

[정규식 참고 사이트](https://docs.python.org/ko/3/library/re.html)


In [ ]:
import re

sentence = "He is a ten-year-old boy."
sentence = re.sub('([^a-zA-Z.,?!])', ' ', sentence)

print(sentence)

He is a ten year old boy.


In [ ]:
# 위에서 배운 내용을 활용하여 문장을 정제해보기

corpus = \
"""
In the days that followed I learned to spell in this uncomprehending way a great many words, among them pin, hat, cup and a few verbs like sit, stand and walk. 
But my teacher had been with me several weeks before I understood that everything has a name.
One day, we walked down the path to the well-house, attracted by the fragrance of the honeysuckle with which it was covered. 
Some one was drawing water and my teacher placed my hand under the spout. 
As the cool stream gushed over one hand she spelled into the other the word water, first slowly, then rapidly. 
I stood still, my whole attention fixed upon the motions of her fingers. 
Suddenly I felt a misty consciousness as of something forgotten—a thrill of returning thought; and somehow the mystery of language was revealed to me. 
I knew then that "w-a-t-e-r" meant the wonderful cool something that was flowing over my hand. 
That living word awakened my soul, gave it light, hope, joy, set it free! 
There were barriers still, it is true, but barriers that could in time be swept away.
""" 

def cleaning_text(text, punc, regex):
  # 노이즈 유형 (1) 문장부호 공백추가
  for p in punc:
    text = text.replace(p, ' ' + p + ' ')
  
  # 노이즈 유형 (2), (3) 소문자화 및 특수문자 제거
  text = re.sub(regex, ' ', text).lower()

  return text

print(cleaning_text(corpus, [',', ',', '!', '?'], '([^a-zA-Z0-9.,?!\n])'))


in the days that followed i learned to spell in this uncomprehending way a great many words  ,   among them pin  ,   hat  ,   cup and a few verbs like sit  ,   stand and walk. 
but my teacher had been with me several weeks before i understood that everything has a name.
one day  ,   we walked down the path to the well house  ,   attracted by the fragrance of the honeysuckle with which it was covered. 
some one was drawing water and my teacher placed my hand under the spout. 
as the cool stream gushed over one hand she spelled into the other the word water  ,   first slowly  ,   then rapidly. 
i stood still  ,   my whole attention fixed upon the motions of her fingers. 
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me. 
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand. 
that living word awakened my soul  ,   gave it light  ,   hope  ,   joy  


1-3. 분산표현 : 바나나와 사과의 관계를 어떻게 표현할까?

1. 단어의 희소 표현과 분산 표현

(1) 희소 표현
- 벡터의 각 차원마다 단어의 특정 의미 속성을 대응시키는 방식이다.
- 예를 들어, 남자와 여자는 남자 : [1], 여자 : [-1]의 형태로 표현 할 수 있다. 그 다음으로 소년, 소녀라면 성별이라는 속성에 나이가 어리다를 추가해야 한다. 소년 : [1,1], 소녀 : [-1,1]
- 이렇게 대상에 따른 속성을 나타내는게 희소 표현입니다.
- 만약 위에 예시에서 바나나를 추가한다면 다음과 같이 만들 수 있습니다. 바나나 : [0,0,1, 0.5]



In [ ]:
# 희소표현 예시
{
          [성별, 연령, 과일, 색깔]
    남자: [-1.0, 0.0, 0.0, 0.0],
    여자: [1.0, 0.0, 0.0, 0.0],
    사과: [0.0, 0.0, 1.0, 0.5],   // 빨갛게 잘 익은 사과
    바나나: [0.0, 0.0, 1.0, -0.5] // 노랗게 잘 익은 바나나
}

SyntaxError: ignored

(2) 분산 표현
- 기본적으로 분포 가설이라는 가정 하에 만들어진 표현 방법
- 이 가정은 비슷한 문맥에서 등장하는 단어들은 비슷한 의미를 가진다.
- 예를들어 강아지라는 단어는 귀엽다, 예쁘다 등의 단어가 주로 함께 등장하는데 분포 가설에 따라서 해당 내용을 가진 텍스트의 단어들을 벡터화한다면 해당 단어 벡터들은 유사한 벡터값을 가진다.

[분산표현 위키독스](https://wikidocs.net/22660)

코사인 유사도

두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도를 의미합니다. 

[코사인 유사도 이론 참고](https://wikidocs.net/24603)

![](https://wikidocs.net/images/page/24603/%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84.PNG)
<center>코사인 유사도 예시</center>

In [ ]:
# 코사인 유사도 계산

import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

word_1 = np.array([-1.0, 0.0, 0.0, 0.0])
word_2 = np.array([0.0, 0.0, 1.0, 0.5])

print('word_1과 word_2의 유사도 :',cos_sim(word_1, word_2))
# word_1과 word_2 공유하는 의미속성이 없는 위 두 벡터의 내적은 0이 되므로, 코사인 유사도도 역시 0이 됩니다.

word_1과 word_2의 유사도 : 0.0


그래서 우린 Embedding 레이어를 사용해 각 단어가 몇 차원의 속성을 가질지 정의하는 방식으로 단어의 분산 표현(distributed representation) 를 구현하는 방식을 주로 사용하게 됩니다. 만약 100개의 단어를 256차원의 속성으로 표현하고 싶다면 Embedding 레이어는 아래와 같이 정의되겠죠.

위 단어의 분산 표현에는 우리가 일일이 정의할 수 없는 어떤 추상적인 속성들이 256차원 안에 골고루 분산되어 표현됩니다. 희소 표현처럼 속성값을 임의로 지정해 주는 것이 아니라, 수많은 텍스트 데이터를 읽어가며 적합한 값을 찾아갈 것입니다. 적절히 훈련된 분산 표현 모델을 통해 우리는 단어 간의 의미 유사도를 계산하거나, 이를 feature로 삼아 복잡한 자연어처리 모델을 훈련시킬 수 있게 됩니다.

In [ ]:
# embedding_layer = tf.keras.layers.Embedding(input_dim=100, output_dim=256)

2. 단어 사전 구성과 활용의 문제

짚고 넘어가야 할 부분이 있다. 위에서 Embedding 레이어를 사용해 구현한 분산 표현은 컴퓨터 입장에서는 단어 사전이 될 것입니다. 하지만 단어 사전만으로는 완전한 해석

컴퓨터 또한 비슷하다. 문장이 단어 단위로 분리되어있어서 각 단어에 해당하는 분산 표현을 언제든 정확히 찾을 수 있을 것처럼 전제하고 위와 같은 컴퓨터용 단어 사전을 구축해 두었지만, 정작 컴퓨넌 전혀 엉뚱한 단어로 해석하거나 혹은 사전에서 그 단어를 찾지 못해 당황하는 일이 생기게 됩니다.그것은 바로 컴퓨터가 문장을 단어 단위로 정확하게 끊어 읽지 못하기 때문에 벌어지는 일입니다. 다음 스텝에서 이 문제의 어려움을 좀 더 자세하게 다루어 봅시다.



1-4. 토큰화 : 그녀는? 그녀+는?

한 문장에서 단어의 수는 어떻게 정의할 수 있을까요? "그녀는 나와 밥을 먹는다" 라는 문장이 주어지면 공백 기준으로 나누어 1: 그녀는 2: 나와 3: 밥을 4: 먹는다 4개 단어로 이루어졌다고 단정 지을 수 있을까요? 어쩌면 1: 그녀 2: 는 3: 나 4: 와 5: 밥 6: 을 7: 먹는다 처럼 잘게 잘게 쪼개어 7개 단어로 이루어졌다고 할 수도 있겠죠? 그것은 우리가 정의할 토큰화 기법이 결정할 부분이랍니다!



1. 공백 기반 토큰화

자연어의 노이즈를 제거하는 방법 중 하나로 우리는 Hi, 를 Hi와 ,로 나누기 위해 문장부호 양옆에 공백을 추가해 주었습니다. 그것은 이 공백 기반 토큰화를 사용하기 위해서였죠! 당시의 예제 코드를 다시 가져와 공백을 기반으로 토큰화를 진행해 보겠습니다.


In [ ]:
corpus = \
"""
in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it free !  
there were barriers still ,  it is true ,  but barriers that could in time be swept away . 
"""

tokens = corpus.split()

print('문장이 포함하는 Tokens', tokens)

문장이 포함하는 Tokens ['in', 'the', 'days', 'that', 'followed', 'i', 'learned', 'to', 'spell', 'in', 'this', 'uncomprehending', 'way', 'a', 'great', 'many', 'words', ',', 'among', 'them', 'pin', ',', 'hat', ',', 'cup', 'and', 'a', 'few', 'verbs', 'like', 'sit', ',', 'stand', 'and', 'walk', '.', 'but', 'my', 'teacher', 'had', 'been', 'with', 'me', 'several', 'weeks', 'before', 'i', 'understood', 'that', 'everything', 'has', 'a', 'name', '.', 'one', 'day', ',', 'we', 'walked', 'down', 'the', 'path', 'to', 'the', 'well', 'house', ',', 'attracted', 'by', 'the', 'fragrance', 'of', 'the', 'honeysuckle', 'with', 'which', 'it', 'was', 'covered', '.', 'some', 'one', 'was', 'drawing', 'water', 'and', 'my', 'teacher', 'placed', 'my', 'hand', 'under', 'the', 'spout', '.', 'as', 'the', 'cool', 'stream', 'gushed', 'over', 'one', 'hand', 'she', 'spelled', 'into', 'the', 'other', 'the', 'word', 'water', ',', 'first', 'slowly', ',', 'then', 'rapidly', '.', 'i', 'stood', 'still', ',', 'my', 'whole', 'attentio

2. 형태소 기반 토큰화

하지만 우리에겐 영어 문장이 아닌 한국어 문장을 처리할 일이 더 많을 것이고, 한국어 문장은 공백 기준으로 토큰화를 했다간 엉망진창의 단어들이 등장하는 것을 알 수 있습니다. 문장부호처럼 "은 / 는 / 이 / 가" 양옆에 공백을 붙이자구요? 글쎄요... 가로 시작하는 단어만 해도 가면, 가위, 가족, 가수... 의도치 않은 변형이 너무나도 많이 일어날 것 같네요!

이를 어떻게 해결할 수 있을까요? 정답은 형태소에 있습니다. 어릴 적 국어 시간에 배운 기억이 새록새록 나시나요? 상기시켜드리면 형태소의 정의는 아래와 같습니다.

(명사) 뜻을 가진 가장 작은 말의 단위.

예를 들어, 오늘도 공부만 한다 라는 문장이 있다면, 오늘, 도, 공부, 만, 한다 로 쪼개지는 것이 바로 형태소죠. 한국어는 이를 활용해 토큰화를 할 수 있습니다!

한국어 형태소 분석기에서 대표적인 KoNLPy를 사용하여 봅시다.

[KoNLPy: 파이썬 한국어 NLP - KoNLPy 0.4.3 documentation](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)

KoNLPy는 내부적으로 5가지의 형태소 분석 Class를 포함하고 있습니다. 형태소 분석기들은 특수한 문장(띄어쓰기 X / 오탈자) 처리 성능, 속도 측면에서 차이를 보입니다. 천하무적인 것은 (아직은) 없으니, 각 분석기를 직접 테스트해보고 적합한 것을 선택해 사용하면 됩니다.

[KoNLPy: 파이썬 한국어 NLP - KoNLPy 0.4.3 documentation](https://iostream.tistory.com/144)

Q6. 아래 질문에 적합한 분석기를 찾아보도록 합시다!

인턴 M: 100만 문장의 데이터를 형태소 분석하고 싶습니다. 속도 측면에서 가장 뛰어난 분석기를 쓰는 게 좋아 보이는데, 어떤 분석기를 사용하는 게 좋을까요? 추천해주세요!
- mecab

Q7. 아래 질문에 적합한 분석기를 찾아보도록 합시다!

팀장 K: 시간은 오래 걸려도 괜찮으니, 우리는 정확도로 승부합시다. 대신 우리가 사용할 SNS 데이터는 맞춤법이 테러블하니까, 띄어쓰기나 오탈자에도 강건한 분석기를 사용해요. 고생~
- KOMORAN + (mecab, 꼬꼬마)




In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
from konlpy.tag import Hannanum,Kkma,Komoran,Mecab,Okt

In [ ]:
# 형태소 성능 시험해보기

tokenizer_list = [Hannanum(), Kkma(), Komoran(), Mecab(), Okt()]

kor_text = '코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤 전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.'

for tokenizer in tokenizer_list:
  print('[{}] \n{}'.format(tokenizer.__class__.__name__, tokenizer.pos(kor_text)))

[Hannanum] 
[('코로나바이러스', 'N'), ('는', 'J'), ('2019년', 'N'), ('12월', 'N'), ('중국', 'N'), ('우한', 'N'), ('에서', 'J'), ('처음', 'M'), ('발생', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('뒤', 'N'), ('전', 'N'), ('세계', 'N'), ('로', 'J'), ('확산', 'N'), ('되', 'X'), ('ㄴ', 'E'), (',', 'S'), ('새롭', 'P'), ('은', 'E'), ('유형', 'N'), ('의', 'J'), ('호흡기', 'N'), ('감염', 'N'), ('질환', 'N'), ('이', 'J'), ('ㅂ니다', 'E'), ('.', 'S')]
[Kkma] 
[('코로나', 'NNG'), ('바', 'NNG'), ('이러', 'MAG'), ('슬', 'VV'), ('는', 'ETD'), ('2019', 'NR'), ('년', 'NNM'), ('12', 'NR'), ('월', 'NNM'), ('중국', 'NNG'), ('우', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('에', 'VV'), ('서', 'ECD'), ('처음', 'NNG'), ('발생', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('뒤', 'NNG'), ('전', 'NNG'), ('세계', 'NNG'), ('로', 'JKM'), ('확산', 'NNG'), ('되', 'XSV'), ('ㄴ', 'ETD'), (',', 'SP'), ('새', 'NNG'), ('롭', 'XSA'), ('ㄴ', 'ETD'), ('유형', 'NNG'), ('의', 'JKG'), ('호흡기', 'NNG'), ('감염', 'NNG'), ('질환', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EFN'), ('.', 'SF')]
[Komoran] 
[('코로나바이러스', 'NNP'), ('는', 'JX'), ('2019', 'SN'

3. 사전에 없는 단어의 문제

지금까지 배운 공백 기반이나, 형태소 기반의 토큰화 기법들은 모두 의미를 가지는 단위로 토큰을 생성합니다. 이 기법의 경우, 데이터에 포함되는 모든 단어를 처리할 수는 없기 때문에 자주 등장한 상위 N개의 단어만 사용하고 나머지는 <unk>같은 특수한 토큰(Unknown Token)으로 치환해버립니다. 자원이 무한하고 데이터도 무한하다면 그러지 않아도 되겠지만... 모든 것은 한정되어 있기 때문에 이 또한 불가피한 손실이죠. 하지만 이것은 종종 큰 문제를 야기합니다.

코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤
전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.

→

<unk>는 2019년 12월 중국 <unk>에서 처음 발생한 뒤
전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.

만약 위 문장을 영문으로 번역해야 한다면 어떨까요? 핵심인 단어 코로나바이러스와 우한을 모른다면 제대로 해낼 수 있을 리가 없습니다. 이를 OOV(Out-Of-Vocabulary) 문제라고 합니다. 이처럼 새로 등장한(본 적 없는) 단어에 대해 약한 모습을 보일 수밖에 없는 기법들이기에, 이를 해결하고자 하는 시도들이 있었습니다. 


1-5. 토큰화 : 다른 방법들

Wordpiece Model(WPM) 은 우리가 접한 적이 있는 아이디어를 기반으로 만들어졌습니다. 두 단어 preview와 predict를 보면 접두어인 pre가 공통되고 있죠? pre가 들어간 단어는 주로 "미리", "이전의" 와 연계되는 의미를 가지고 있습니다. 컴퓨터도 두 단어를 따로 볼 게 아니라 pre+view와 pre+dict로 본다면 학습을 더 잘 할 수 있지 않을까요?

이처럼 한 단어를 여러 개의 Subword의 집합으로 보는 방법이 WPM입니다. WPM의 원리를 알기 전, 먼저 알아야 할 것이 바로 Byte Pair Encoding(BPE) 입니다.

1. Byte Pair Encoding(BPE)

BPE 알고리즘이 고안된 것은 1994년입니다. 그때는 자연어 처리에 적용하기 위해서가 아니라 데이터 압축을 위해서 생겨났었죠. 데이터에서 가장 많이 등장하는 바이트 쌍(Byte Pair) 을 새로운 단어로 치환하여 압축하는 작업을 반복하는 방식으로 동작합니다. 예시는 아래와 같습니다.

[Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/pdf/1508.07909.pdf)

In [ ]:
aaabdaaabac # 가장 많이 등장한 바이트 쌍 "aa"를 "Z"로 치환합니다.
→ 
ZabdZabac   # "aa" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.
Z=aa        # 그다음 많이 등장한 바이트 쌍 "ab"를 "Y"로 치환합니다.
→ 
ZYdZYac     # "ab" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.
Z=aa        # 여기서 작업을 멈추어도 되지만, 치환된 바이트에 대해서도 진행한다면
Y=ab        # 가장 많이 등장한 바이트 쌍 "ZY"를 "X"로 치환합니다.
→ 
XdXac
Z=aa
Y=ab
X=ZY       # 압축이 완료되었습니다!

In [ ]:
# Neural Machine Translation of Rare Words with Subword Units 논문 파이썬 구현
import re, collections
# 임의의 데이터에 포함된 단어들입니다.
# 우측의 정수는 임의의 데이터에 해당 단어가 포함된 빈도수입니다.
vocab = {
    'l o w '      : 5,
    'l o w e r '  : 2,
    'n e w e s t ': 6,
    'w i d e s t ': 3
}

num_merges = 5

def get_stats(vocab):
  """
  단어 사전을 불러와
  단어는 공백 단위로 쪼개어 문자 list를 만들고
  빈도수와 쌍을 이루게 합니다. (symbols)
  """
  pairs = collections.defaultdict(int)

  for word, freq in vocab.items():
    symbols = word.split()

    for i in range(len(symbols) - 1): # 모든 symbols를 확인하여
      pairs[symbols[i], symbols[i+1]] += freq # 문자 쌍의 빈도수를 저장

  return pairs

def merge_vocab(pair, v_in):
  v_out = {}
  bigram = re.escape(' '.join(pair))
  p = re.compile(r'(?<!\S)' + bigram + '(?<!\S)')

  for word in v_in:
    w_out = p.sub(''.join(pair), word)
    v_out[w_out] = v_in[word]

  return v_out, pair[0] + pair[1]

token_vocab = []

for i in range(num_merges):
  print('>> Step {0}'.format(i+1))

  pairs = get_stats(vocab)
  best = max(pairs, key=pairs.get) # 가장 많은 빈도수를 가진 문자 쌍을 반환합니다.
  vocab, merge_tok = merge_vocab(best, vocab)
  print("다음 문자 쌍을 치환:", merge_tok)
  print("변환된 Vocab:\n", vocab, "\n")
  token_vocab.append(merge_tok)

print("Merge Vocab:", token_vocab)

>> Step 1
다음 문자 쌍을 치환: es
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w e s t ': 6, 'w i d e s t ': 3} 

>> Step 2
다음 문자 쌍을 치환: es
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w e s t ': 6, 'w i d e s t ': 3} 

>> Step 3
다음 문자 쌍을 치환: es
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w e s t ': 6, 'w i d e s t ': 3} 

>> Step 4
다음 문자 쌍을 치환: es
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w e s t ': 6, 'w i d e s t ': 3} 

>> Step 5
다음 문자 쌍을 치환: es
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w e s t ': 6, 'w i d e s t ': 3} 

Merge Vocab: ['es', 'es', 'es', 'es', 'es']


만일 lowest라는 처음 보는 단어가 등장하더라도, 위 알고리즘을 따르면 어느 정도 의미가 파악된 low와 est의 결합으로 표현할 수 있습니다. 또 BPE의 놀라운 점은 아무리 큰 데이터도 원하는 크기로 OOV 문제없이 사전을 정의할 수 있다는 것입니다. 극단적으로 생각했을 때 알파벳 26개와 특수문자, 문장부호를 아무리 추가해도 100개 이내로 사전을 정의할 수 있죠.(물론 그러면 안 됩니다!!)

Embedding 레이어는 단어의 개수 x Embedding 차원 수 의 Weight를 생성하기 때문에 단어의 개수가 줄어드는 것은 곧 메모리의 절약으로 이어집니다. 많은 데이터가 곧 정확도로 이어지기 때문에 이런 기여는 굉장히 의미가 있습니다!

하지만 아직도! 완벽하다고는 할 수 없습니다. 만약 수많은 데이터를 사용해 만든 BPE 사전으로 모델을 학습시키고 문장을 생성하게 했다고 합시다. 그게 [i, am, a, b, o, y, a, n, d, you, are, a, gir, l]이라면, 어떤 기준으로 이들을 결합해서 문장을 복원하죠? 몽땅 한꺼번에 합쳤다간 끔찍한 일이 벌어질 것만 같습니다...

2. Wordpiece Model(WPM)

이에 구글에서 BPE를 변형해 제안한 알고리즘이 바로 WPM입니다. WPM은 BPE에 대해 두 가지 차별성을 가집니다.

(1) 공백 복원을 위해 단어의 시작 부분에 언더바 _ 를 추가합니다.

(2) 빈도수 기반이 아닌 가능도(Likelihood)를 증가시키는 방향으로 문자 쌍을 합칩니다.

첫 번째 문항은 아주 쉬운 내용으로, 앞서 사용한 예문을 빌리면 [_i, _am, _a, _b, o, y, _a, n, d, _you, _are, _a, _gir, l]로 토큰화를 한다는 것입니다. 이렇게 하면 문장을 복원하는 과정이 1) 모든 토큰을 합친 후, 2) 언더바 _를 공백으로 치환으로 마무리되어 간편하죠.

두 번째 문항은 다소 난해하게 다가올 수도 있습니다. 직관적인 이해를 얻고 넘어가는 것을 목표로 하죠. 본 내용은 아래 논문 3절과 4절에 자세하게 나와 있습니다.

[JAPANESE AND KOREAN VOICE SEARCH](https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/37842.pdf)

구글은 이 기법을 한국어, 일본어 텍스트 처리를 위해 고려했고 2가지 시사점이 있다.

(1) 조사, 어미 등의 활용이 많고 복잡한 한국어 같은 모델의 토크나이저로 WPM이 좋은 대안이 될 수 있다.

(2) WPM은 어떤 언어든 무관하게 적용 가능한 language-neutral하고 general한 기법이다. 한국어 형태소 분석기처럼 한국어에만 적용 가능한 기법보다 훨씬 활용도가 크다

[확률(probability)과 가능도(likelihood) 그리고 최대우도추정(likelihood maximazation)](https://jjangjjong.tistory.com/41)

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-1-L-3.max-800x600_uZeTAWo.png)
<center>Katz's Back-off Model </center>

위 수식은 그냥 넘어가도 상관없다.

토큰화의 끝판왕으로 보이는 이 WPM은 아쉽게도 공개되어 있지는 않습니다. 대신에 구글의 SentencePiece 라이브러리를 통해 고성능의 BPE를 사용할 수 있습니다! SentencePiece에는 전처리 과정도 포함되어 있어서, 데이터를 따로 정제할 필요가 없어 간편하기까지 합니다.

[google/sentencepiece](https://github.com/google/sentencepiece)

3. soynlp

이외에도 한국어를 위한 토크나이저로 soynlp를 활용할 수 있습니다. soynlp는 한국어 자연어 처리를 위한 라이브러리인데요. 토크나이저 외에도 단어 추출, 품사 판별, 전처리 기능도 제공합니다.

[soynlp](https://github.com/lovit/soynlp)

형태소 기반인 konlpy의 단점을 해결하기 위해 사용됩니다.

soynlp의 토크나이저는 '학습데이터를 이용하지 않으면서 데이터에 존재하는 단어를 찾거나, 문장을 단어열로 분해, 혹은 품사 판별을 할 수 있는 비지도학습 접근법을 지향합니다'라고 한다.

문장에서 처음 단어를 받아들일 때 단어의 경계를 알아야 올바르게 토큰화를 할 수 있습니다. 이때 단어의 경계를 비지도학습을 통해 결정하겠다는 말이에요. 비지도학습을 통한 방법이기 때문에 미등록 단어도 토큰화가 가능합니다. 여기서 비지도학습을 가능케 하는 것이 통계적인 방법이라서 soynlp를 통계 기반 토크나이저로 분류하기도 합니다.

트와이스가 한 단어임을 인지하기 위해서 트, 트와, 트와이, 트와이스 각각 다음 글자의 확률을 계산해서 비교한다고 생각하면 좋습니다. 수학적으로 자세한 내용은 여기서는 다루지 않습니다. 다만 koNLPy외에도 soynlp가 있다는 점을 기억해 주세요.


# 1-6. 토큰에게 의미를 부여하기

각 토큰들이 랜덤하게 부여된 실수로 살아가지 않게, 그들끼리 유사도 연산을 할 수 있게 의미를 부여하는 알고리즘, 당연히 있습니다! 심지어 토큰화 기법보다 더 많이요! 대표적인 3가지만 알아보도록 하겠습니다. 이번 코스에선 간단하게 아이디어만 이해해도 충분합니다.

1. Word2Vec

[위키독스/Word2Vec](https://wikidocs.net/22660)

Word2Vec은 단어를 벡터로 만든다는 의미

난 오늘 술을 한 잔 마셨어 라는 문장의 각 단어 즉, 동시에 등장하는 단어끼리는 연관성이 있다는 아이디어로 시작된 알고리즘입니다. 예문의 경우 다른 단어는 몰라도 술과 마셨어는 괜찮은 연관성을 캐치해낼 수 있겠네요.

Q8. Word2Vec에는 두 가지 방식이 존재합니다. 각각 무엇인가요?

CBOW(Continuous Bag of Words)
- 주변에 있는 단어들을 입력으로 중간에 있는 단어들을 예측하는 방법

Skip-Gram
- 중간에 있는 단어들을 입력으로 주변 단어들을 예측하는 방법

Q9. 윗글의 저자에 따르면 Word2Vec은 딥러닝 모델이 아닙니다. 그 이유는 무엇일까요?
- Word2Vec은 은닉층이 다수인 딥 러닝(deep learning) 모델이 아니라 은닉층이 1개인 얕은 신경망(shallow neural network)이라는 점입니다.

Q10. Q8의 두 가지 방식 중, 어느 것이 더 성능이 좋을 것 같은지 생각해봅시다. 그리고 실제로는 어떤지 인터넷에서 찾아 적어봅시다.
- 여러 논문에서 성능 비교를 진행했을 때 전반적으로 Skip-gram이 CBOW보다 성능이 좋다고 알려져 있습니다.

2. FastText

Word2Vec은 좋은 방법이지만, 연산의 빈부격차가 존재한다. 자주 등장하지 않는 단어는 최악의 경우 단 한 번의 연산만을 거쳐 랜덤하게 초기화된 값과 크게 다르지 않은 상태로 알고리즘이 종료될 수 있습니다. FastText는 이를 해결하기 위해 BPE와 비슷한 아이디어를 적용했습니다.

[한국어를 위한 어휘 임베딩의 개발 -1-](https://brunch.co.kr/@learning/7)

Q11. 기존의 단어마다 Embedding을 할당하던 방식과 FastText의 큰 차이점은 무엇인가요? 간단하게 서술해봅시다
- FastText는 위에서 언급한 SGNS 방법에 기반을 둔다.

SGNS란?
- Skip-Gram with Negative Sampling으로 2013~4년 이후로 등장한 논문들을 말한다.

SGNS와 유일하게 달라지는 점은 텍스트의 최소 단위를 '어휘'가 아닌 '어위를 구성하는 글자 n-gram'단위로 한 수준 내렸다.

즉 기존에는 임베딩 벡터를 어휘마다 하나씩 할당하고 이를 학습했다면, 이제 어휘를 구성하고 있는 n-gram마다 하나씩 이를 할당하고, 어휘를 구성하는 모든 n-gram 벡터의 평균 벡터를 어휘 임베딩으로 본 것입니다.


ELMo - the 1st Contextualized Word Embedding

위에서 소개한 알고리즘들도 전부 성능은 좋지만 고정적이라는 문제를 가지고있다. 즉 동음이의어를 처리할 수 없다는 얘기이다.

예시)
이렇게나 탐스럽고 먹음직스러운 사과를 보셨나요?

저의 간절한 사과를 받아주시기 바랍니다.

두 문장에서 사과의 뜻은 다르다. 그러나 Word2Vec, FastText이든 간에 이 두 문장에 나오는 사과의 워드 벡터값은 동일할 수밖에 없다.

사과의 의미를 명확하게 나누기 위해서는 Context-sensitive Grammar를 따르는 자연어를 이해하려면 문맥(context)의 활용이 필수적입니다. 단어를 이해하기위해 그 단어가 놓인 주변 단어 배치의 맥락이 함께 고려되는 WordEmbedding이 필요한 것입니다.

2018년 NLP계에 큰 폭풍을 몰고 왔던 ELMo라는 모델은 데이터에 단어가 등장한 순간, 그 주변 단어 정보를 사용해 Embedding을 구축하는 개념을 처음 소개하면서 자연어처리의 획기적인 발전의 계기를 마련해 준 첫 번째 Contextualized Word Embedding 모델입니다. 

[전이 학습 기반 NLP (1):ELMo](https://brunch.co.kr/@learning/12)

Q12. 양방향 LSTM을 활용한 ELMo 모델에서 Contextual Word Embedding이 되는 벡터는 어떤 벡터 3가지를 합쳐서 얻어지나요?
- 기존 어휘 임베딩(노란색), 순방향 LSTM의 잠재 상태(왼쪽 파란색), 역방향 LSTM의 잠재 상태(오른쪽 파란색)을 모두 종합하여 이어 붙인 벡터가 ELMo 임베딩이 됩니다.


1-7. 마무리

이번에 배운 내용은 토큰화와 분산 표현이 중심입니다. 문장이 입력되면 적절히 토큰화를 하고 토큰을 임베딩(Embedding)을 통해 분산 표현으로 만드는 것이지요. 분산 표현은 벡터이므로 이제 인공지능에 활용할 수 있습니다.

토큰화에 사용되는 방법은 언어마다 다른데요. 문장 구성 성분이 다르기 때문입니다. 조사가 있는 한국어는 형태소 기반인 koNLPy를 주로 쓰고, WordPiece Model인 SentencePiece를 쓸 수도 있어요. 물론 그 외에 다른 방법도 있습니다.

토큰화를 마친 후 임베딩을 할 때는 토큰마다 독립적으로 만들지 않고 토큰 간의 관계성을 주입합니다. 그래야 문장을 구성할 때 적절히 사용될 수 있기 때문이에요. 이렇게 토큰 간의 관계성을 고려해서 만든 것으로는 Word2Vec, FastText 등이 있어요. 거기다가 문장의 문맥까지 고려하는 ELMo까지 등장했습니다.

이제 지금까지 배운 내용이 그려지나요? 오늘 배운 것들은 앞으로 진행될 모든 것들의 기초가 되는 부분이니 꼭 숙지하시길 권장합니다.